Referred doc-https://huggingface.co/docs/transformers/tasks/sequence_classification


- Show the runtime used (T4)
- Upload the financial_sentiment_data.csv file to Colab

In [1]:
pip install numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 37.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [3]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00


In [1]:
import numpy as np
print(np.__version__)

1.26.4


Importing required libraries

In [2]:
import numpy as np
import pandas as pd

https://www.kaggle.com/datasets/sbhatti/financial-sentiment-analysis?select=data.csv

In [4]:
financial_data = pd.read_csv("/content/financial_sentiment_data.csv")

financial_data.sample(10)

,Sentence,Sentiment
877,Our customers include companies in the energy ...,neutral
2314,The financial details of the acquisition were ...,neutral
2067,"Risto Jalo , chief executive of Dormus Print a...",neutral
4023,Earnings per share ( EPS ) amounted to EUR1 .3...,neutral
4783,The new activity will incur an investment of a...,neutral
2641,"The operating loss amounted to EUR 0.8 mn , co...",negative
2330,Subscription sales decreased slightly .,negative
430,Finnish fibers and plastic products maker Suom...,positive
2524,These sections will be put into place to form ...,neutral
5110,"The Daily Graphic newspaper , in October , rep...",neutral


Checking dimension of data

In [5]:
financial_data.shape

(5842, 2)

In [6]:
financial_data["Sentiment"].value_counts()

,count
Sentiment,
neutral,3130
positive,1852
negative,860


In [7]:
financial_data["label"] = financial_data["Sentiment"].replace({"negative": 0, "neutral": 1, "positive": 2})

financial_data.sample(10)

<ipython-input-7-619f1b1d93a1>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  financial_data["label"] = financial_data["Sentiment"].replace({"negative": 0, "neutral": 1, "positive": 2})


,Sentence,Sentiment,label
3532,` By separating side businesses we will be abl...,positive,2
1519,The number of class A shares remains unchanged...,neutral,1
4467,`` The Intel Atom processor has had tremendous...,positive,2
4694,Operating cash flow after investments totalled...,neutral,1
1570,Out of shorts $ALTR at 38.85 with loss,positive,2
349,Full-year operating result for 2008 was 3.6 mi...,negative,0
4518,TELE2 Affarsvarlden gave a `` buy '' recommend...,positive,2
2832,"When dialing in , the participants should quot...",neutral,1
3370,$FB (110.20) is starting to show some relative...,positive,2
3923,Operating profit rose from EUR 1.94 mn to EUR ...,positive,2


Converting pandas df to huggingface dataset

In [8]:
from datasets import Dataset

financial_ds = Dataset.from_pandas(financial_data)

financial_ds

Dataset({
    features: ['Sentence', 'Sentiment', 'label'],
    num_rows: 5842
})

Casts the given column as :obj:datasets.features.ClassLabel and updates the table.

In [9]:
financial_ds = financial_ds.class_encode_column("label")

Stringifying the column:   0%|          | 0/5842 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/5842 [00:00<?, ? examples/s]

Splitting the dataset’s  into a train and test set with the train_test_split method

In [10]:
financial_ds = financial_ds.train_test_split(test_size = 0.2, stratify_by_column = 'label', seed = 123)

financial_ds

DatasetDict({
    train: Dataset({
        features: ['Sentence', 'Sentiment', 'label'],
        num_rows: 4673
    })
    test: Dataset({
        features: ['Sentence', 'Sentiment', 'label'],
        num_rows: 1169
    })
})

Negative, positive , neutral instances can be seen

In [11]:
financial_ds['train'][1]

{'Sentence': '$YHOO A breakout above $29.83 would constitute a technical entry for the short term trader. http://stks.co/jkUF',
 'Sentiment': 'positive',
 'label': 2}

In [12]:
financial_ds['train'][2]

{'Sentence': 'New Chairman of the Board of Directors , Mr Chaim Katzman , will give a presentation and answer questions .',
 'Sentiment': 'neutral',
 'label': 1}

In [13]:
financial_ds['train'][26]

{'Sentence': '@chessNwine: $IWM 30-Minute Chart. Small caps threatening descending triangle breakdown under $110.20.  http://stks.co/r0KKm',
 'Sentiment': 'negative',
 'label': 0}

https://huggingface.co/roberta-base

NOTE that this is a model pre-trained using masked language modeling (MLM) and we are going to fine-tune it on a classification task

Here we will set up a token to access the Hugging Face hub

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Creating a preprocessing function to tokenize text and truncate sequences to be no longer than roberta-base maximum input length (512 tokens)

In [15]:
def preprocess_function(examples):
    return tokenizer(examples["Sentence"], truncation = True)

In [16]:
tokenizer.vocab_size

50265

To apply the preprocessing function over the entire dataset, use 🤗 Datasets map function. You can speed up map by setting batched=True to process multiple elements of the dataset at once:

In [17]:
tokenized_financial_ds = financial_ds.map(preprocess_function, batched = True)

Map:   0%|          | 0/4673 [00:00<?, ? examples/s]

Map:   0%|          | 0/1169 [00:00<?, ? examples/s]

In [18]:
financial_ds["train"][45]

{'Sentence': '( ADP News ) - Feb 4 , 2009 - Finnish broadband data communication systems and solutions company Teleste Oyj ( HEL : TLT1V ) said today its net profit decreased to EUR 5.5 million ( USD 7.2 m ) for 2008 from EUR 9.4 million for 200',
 'Sentiment': 'negative',
 'label': 0}

Tokenizing that instance

In [19]:
tokenized_financial_ds["train"][45]

{'Sentence': '( ADP News ) - Feb 4 , 2009 - Finnish broadband data communication systems and solutions company Teleste Oyj ( HEL : TLT1V ) said today its net profit decreased to EUR 5.5 million ( USD 7.2 m ) for 2008 from EUR 9.4 million for 200',
 'Sentiment': 'negative',
 'label': 0,
 'input_ids': [0,
  1640,
  4516,
  510,
  491,
  4839,
  111,
  1927,
  204,
  2156,
  2338,
  111,
  21533,
  11451,
  414,
  4358,
  1743,
  8,
  2643,
  138,
  5477,
  13967,
  17311,
  267,
  36,
  39509,
  4832,
  27017,
  565,
  134,
  846,
  4839,
  26,
  452,
  63,
  1161,
  1963,
  8065,
  7,
  10353,
  195,
  4,
  245,
  153,
  36,
  6775,
  262,
  4,
  176,
  475,
  4839,
  13,
  2266,
  31,
  10353,
  361,
  4,
  306,
  153,
  13,
  1878,
  2],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1

Detokenizing the text to get back the input text

In [20]:
tokenizer.decode(tokenized_financial_ds["train"][45]["input_ids"])

'<s>( ADP News ) - Feb 4 , 2009 - Finnish broadband data communication systems and solutions company Teleste Oyj ( HEL : TLT1V ) said today its net profit decreased to EUR 5.5 million ( USD 7.2 m ) for 2008 from EUR 9.4 million for 200</s>'

Now Creating  a batch of examples using DataCollatorWithPadding. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In NLP, when processing sequences of text (like sentences or paragraphs), it's common to batch these sequences together for efficient training. However, since these sequences can vary in length, DataCollatorWithPadding automatically pads all sequences in a batch to match the length of the longest sequence in that batch. This ensures that all sequences in a batch have the same length, which is a requirement for most deep learning models.

In [21]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer = tokenizer, return_tensors = "tf")

Including a metric during training is often helpful for evaluating your model’s performance. We can quickly load a evaluation method with the 🤗 Evaluate library. For this task, loading the accuracy metric.

In [22]:
import evaluate

accuracy = evaluate.load("accuracy")

Then creating a function that passes your predictions and labels to compute to calculate the accuracy

In [23]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    return accuracy.compute(predictions = predictions, references = labels)

Before you start training your model, creating a map of the expected ids to their labels with id2label and label2id.

In [24]:
id2label = {0: "negative", 1: "neutral", 2: "positive"}

label2id = {"negative": 0, "neutral": 1, "positive": 2}

For Finetuning a model in TensorFlow, first we start by setting up an optimizer function, learning rate schedule, and some training hyperparameters.

create_optimizer creates an AdamWeightDecay optimizer

In [25]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5

batches_per_epoch = len(tokenized_financial_ds["train"]) // batch_size

total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(init_lr = 2e-6, num_warmup_steps = 0, num_train_steps = total_train_steps)

In [26]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "roberta-base", num_labels = 3, id2label = id2label, label2id = label2id
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

Converting train and validation datasets to the tf.data.Dataset format with prepare_tf_dataset():

In [27]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_financial_ds["train"],
    shuffle = True,
    batch_size = 16,
    collate_fn = data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_financial_ds["test"],
    shuffle = False,
    batch_size = 16,
    collate_fn = data_collator,
)

Configuring the model for training with compile. Note that Transformers models all have a default task-relevant loss function, so you don’t need to specify one unless you want to

In [28]:
import tensorflow as tf

model.compile(optimizer = optimizer)

The last two things to setup before we start training is to compute the accuracy from the predictions, done by using Keras callbacks.

Passing  compute_metrics function to KerasMetricCallback:

In [29]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn = compute_metrics, eval_dataset = tf_validation_set)

Write permission is validated by token
- After running the notebook cell - go to https://huggingface.co/
- Go to the top-right corner click on account avatar -> Settings
- Go to Access Tokens - create a Write token
- Copy and paste that in here



In [30]:
from huggingface_hub import notebook_login

notebook_login()

Starting training our model,Calling  fit with your training and validation datasets, the number of epochs, and your callbacks to finetune the model:

In [31]:
model.fit(
    x = tf_train_set, validation_data = tf_validation_set,
    epochs = 2, callbacks = [metric_callback]
)

Epoch 1/2
292/292 [==============================] - 133s 308ms/step - loss: 0.9432 - val_loss: 0.7905 - accuracy: 0.6407
Epoch 2/2
292/292 [==============================] - 72s 247ms/step - loss: 0.6037 - val_loss: 0.4674 - accuracy: 0.7938


https://huggingface.co/jinxxx123

In [32]:
model.push_to_hub("financial_text_sentiment_classification_model")

tf_model.h5:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [33]:
tokenizer.push_to_hub("financial_text_sentiment_classification_model")

README.md:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jinxxx123/financial_text_sentiment_classification_model/commit/a6e225bfaf2aa6b64e465578e9f1e89d1db99380', commit_message='Upload tokenizer', commit_description='', oid='a6e225bfaf2aa6b64e465578e9f1e89d1db99380', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jinxxx123/financial_text_sentiment_classification_model', endpoint='https://huggingface.co', repo_type='model', repo_id='jinxxx123/financial_text_sentiment_classification_model'), pr_revision=None, pr_num=None)


- Go back to the model and show that the model has been pushed (make sure you refresh)
- Show the  model card, files and versions (should NOT be empty), community, and settings

In [34]:
from transformers import pipeline

text = """
  The stock market experienced significant volatility today, with the Dow Jones Industrial Average
  dropping over 500 points in early trading before recovering slightly by the close of the session.
"""

classifier = pipeline("sentiment-analysis", model = model, tokenizer = tokenizer)

classifier(text)

Device set to use 0


[{'label': 'negative', 'score': 0.5118897557258606}]

In [35]:
text = """
  The central bank announced a cut in interest rates, aiming to stimulate economic
  growth by making borrowing cheaper for both consumers and businesses.
"""

classifier(text)

[{'label': 'positive', 'score': 0.7282232642173767}]

In [36]:
text = """
  Inflation rates have been steadily increasing, prompting concerns among economists about
  the potential for reduced purchasing power and higher costs of living.
"""

classifier(text)

[{'label': 'neutral', 'score': 0.4208616614341736}]

In [37]:
text = "The financial advisor recommended a balanced approach to investing, combining both growth and income strategies."

classifier = pipeline("sentiment-analysis", model = "jinxxx123/financial_text_sentiment_classification_model")

classifier(text)

config.json:   0%|          | 0.00/781 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/499M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at jinxxx123/financial_text_sentiment_classification_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Device set to use 0


[{'label': 'neutral', 'score': 0.8468072414398193}]

In [38]:
text = "Strong earnings reports drive the stock to new highs, boosting investor confidence."

classifier(text)

[{'label': 'positive', 'score': 0.8716616630554199}]